# <strong>Gesture Annotation Notebook |</strong>

##### <strong>Author:</strong> <u>Walter Dych</u> <em>(walterpdych@gmail.com)</em>
##### <strong>Edits/Documentation:</strong> <u>Karee Garvin</u> <em>(kgarvin@fas.harvard.edu)</em>

### <strong>Requirements</strong>
- `pandas`
- `numpy`
- `plotly`

### <strong>Generate Annotations</strong>

The `generate_annotations` function takes a pandas DataFrame containing the time, unsmoothed speed, and smoothed speed of a gesture as input and returns a new DataFrame with additional columns that represent annotations for the gesture. The annotations are generated based on the smoothed speed of the gesture and include stroke phase annotations and apex annotations.

The function first initializes several variables and then iterates over the smoothed speed data to generate the annotations. The annotations are generated using a series of if statements that check the current value of the smoothed speed and the previous value of the smoothed speed. The function also keeps track of the start and end indices of each steady phase of the gesture and calculates the index of the minimum value of the smoothed speed within each steady phase to generate the apex annotations.

Finally, the function creates a new DataFrame with the annotations and maps the stroke phase annotations to 'Stroke' or 'Rest'.

In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

In [ ]:
data = pd.read_csv('C:/Users/cosmo/Desktop/Random Scripts/Co-Speech Gesture Automation/Co-Speech-Gesture-Automation/SPEED_FILES/5003_I_right_wrist_processed_data.csv')
data.head()

### <strong>Variables</strong>

- `MAX_S_LENGTH`: an integer that represents the maximum length of an 'S' stroke in the gesture.
- `MAX_STEADY_LENGTH`: an integer that represents the maximum length of a steady phase in the gesture.
- `THRESHOLD`: a float that represents the threshold value for the smoothed speed of the gesture.
- `COLORS`: a dictionary that maps stroke phase annotations to RGBA color values.

These variables are used in the `generate_annotations` function to generate annotations for a gesture based on the smoothed speed of the gesture. The `MAX_S_LENGTH` and `MAX_STEADY_LENGTH` variables are used to determine the maximum length of an 'S' stroke and a steady phase, respectively. The `THRESHOLD` variable is used to determine when the gesture starts or ends a stroke or enters a steady phase. The `COLORS` variable is used to map stroke phase annotations to colors for visualization purposes.

Overall, this excerpt defines some important parameters for generating gesture annotations and demonstrates how variables can be defined and used in Python code.

In [ ]:
MAX_S_LENGTH = 100
MAX_STEADY_LENGTH = 100
THRESHOLD = 0.4
COLORS = {'Stroke': 'rgba(231,107,243,0.2)', 'Rest': 'rgba(107,174,214,0.2)', '': 'rgba(255,255,255,0.2)'}

In [ ]:
def generate_annotations(data): # Generate annotations based on speed_smooth data.
    speed_smooth = list(map(float, data['speed_smooth']))
    phase = [''] * len(speed_smooth)
    last_stroke_type = None
    start_index = None
    steady_start = None
    steady_end = None
    apexes = [''] * len(speed_smooth)

    # Iterate over the speed_smooth array to generate the annotations
    for i in range(1, len(speed_smooth)):
        if speed_smooth[i] > THRESHOLD and speed_smooth[i-1] <= THRESHOLD:
            if last_stroke_type is None or last_stroke_type == 'Steady' or last_stroke_type == 'R':
                last_stroke_type = 'Approach'
                start_index = i
            elif last_stroke_type == 'Approach' and i - start_index >= MAX_S_LENGTH:
                last_stroke_type = None
        elif speed_smooth[i] <= THRESHOLD and speed_smooth[i-1] > THRESHOLD:
            if last_stroke_type == 'Approach':
                last_stroke_type = 'Steady'
                start_index = i
                steady_start = i
            elif last_stroke_type == 'R':
                last_stroke_type = None
                start_index = None
        elif last_stroke_type == 'Steady' and speed_smooth[i] > THRESHOLD and speed_smooth[i-1] <= THRESHOLD:
            if steady_end is not None:
                last_stroke_type = 'R'
                start_index = i

        if last_stroke_type == 'Steady' and steady_start is None:
            steady_start = i
        elif steady_start is not None and last_stroke_type != 'Steady':
            steady_end = i
            min_speed_index = np.argmin(speed_smooth[steady_start:i]) + steady_start
            apexes[min_speed_index] = 'AX'
            steady_start = None
            if speed_smooth[i] > THRESHOLD and speed_smooth[i-1] <= THRESHOLD:
                last_stroke_type = 'R'
                start_index = i
        elif last_stroke_type == 'Steady' and i - start_index >= MAX_STEADY_LENGTH:
            last_stroke_type = None
            start_index = None
            steady_start = None
        phase[i] = last_stroke_type if last_stroke_type is not None else ''

    data = pd.DataFrame({
        'time_ms': data['time_ms'],
        'speed_unsmooth': data['speed_unsmooth'],
        'speed_smooth': data['speed_smooth'],
        'phase': phase, 
        'apex': apexes
    })

    data['phase'] = data['phase'].apply(lambda x: 'Stroke' if x in ['Steady', 'Approach'] else ('Rest' if x == 'R' else ''))
    return data



### <strong>Save Data</strong>

In [ ]:
data = generate_annotations(data)
data.to_csv('C:/Users/cosmo/Desktop/Random Scripts/Co-Speech Gesture Automation/Co-Speech-Gesture-Automation/ANNOTATIONS/5003_I_right_wrist_annotations.csv', index=False)
data.head()

### <strong>Visualize Annotations</strong>
The code first creates a `plotly` figure object and adds a scatter plot of the smoothed speed of the gesture over time. The code then adds rectangles to the plot to represent the stroke phases of the gesture. The rectangles are created by grouping the annotations by phase and finding the start and end times of each phase. The `COLORS` dictionary is used to map the stroke phase annotations to colors for the rectangles.

The code then adds scatter plots to the figure for each stroke phase and for the apex annotations. The `COLORS` dictionary is used to set the colors of the stroke phase markers.

Finally, the code updates the layout of the figure to include axis titles, a title, and a legend. The size of the figure is also set. The figure is then displayed using the `show` method.

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=data['time_ms'], 
    y=data['speed_smooth'], 
    mode='lines',
    name='Speed curve'
))

data['phase_change'] = data['phase'].shift() != data['phase']
data['Group'] = data['phase_change'].cumsum()
interval_df = data.groupby(['Group', 'phase']).agg(Start=('time_ms', 'min'), End=('time_ms', 'max')).reset_index()

shapes = []
for _, row in interval_df.iterrows():
    shapes.append(dict(
        type="rect",
        xref="x", yref="paper",
        x0=row['Start'], x1=row['End'],
        y0=0, y1=1,
        fillcolor=COLORS[row['phase']],
        opacity=0.85,
        layer="below",
        line_width=0,
    ))

for phase in COLORS.keys():
    fig.add_trace(go.Scatter(
        x=[None], y=[None],
        mode='markers',
        marker=dict(size=15, color=COLORS[phase]),
        showlegend=True,
        name=phase,
    ))

apex_df = data[data['apex'] == 'AX']
fig.add_trace(go.Scatter(
    x=apex_df['time_ms'], 
    y=apex_df['speed_smooth'], 
    mode='markers',
    name='Apexes'
))

fig.update_xaxes(title_text='Time (ms)')
fig.update_yaxes(title_text='Speed (px/ms)')
fig.update_layout(showlegend=True, shapes=shapes)
fig.update_layout(title_text='Smoothed Speed Curve with Annotations')
fig.update_layout(width=1250, height=750)
fig.show()
